# Setup spark

In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] ="pyspark-shell"

from pyspark.sql import SparkSession
from pyspark.sql.functions import flatten
from pyspark.sql.types import (StructType, StructField, StringType, 
                                FloatType, DateType, IntegerType, ArrayType)
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .config("spark.driver.memory", "12g") \
        .appName("BDA assignment") \
        .getOrCreate()

# Imports

In [2]:
from typing import NamedTuple, Final, List
#from lxml import etree
import xml.etree.ElementTree as ET
from itertools import islice, chain, combinations
import argparse
import traceback
import bleach
import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.util import pr
#from pyspark.ml.feature import StopWordsRemover
import string
import random
import hashlib
import pandas as pd
import numpy as np
import hashlib
import json
import sys
from itertools import islice

# Constants

In [3]:
SHINGLE_SIZE: Final = 5
SAMPLES: Final = 1000

In [4]:
class comment_tuple(NamedTuple):
    id: int
    #owner_id: int
    post_type: int
    score: int
    text: str

class shingle_set(NamedTuple):
    id: int
    shingles: frozenset[tuple]

class similarity(NamedTuple):
    id_set1: int
    id_set2: int
    similarity: float

# Read and clean XML

In [5]:
def set_schema():
    """
    Define the schema for the DataFrame
    """
    schema_list = []
    schema_list.append(StructField("Id", IntegerType(), True))
    #schema_list.append(StructField("PostTypeId", IntegerType(), True))
    #schema_list.append(StructField("Score", IntegerType(), True))
    schema_list.append(StructField("Body", StringType(), True))
    
    return StructType(schema_list)

def parse_post(rdd):
    results = []
    root = ET.fromstring(rdd[0])

    for elem in root.findall('row'):
        rec = []
        #print("Found row")
        assert elem.text is None, "The row wasn't empty"
        rec.append(int(elem.attrib["Id"]))
        #int(elem.attrib["OwnerUserId"]),
        #rec.append(int(elem.attrib["PostTypeId"])),
        #rec.append(int(elem.attrib["Score"])),
        rec.append(bleach.clean(elem.attrib["Body"], strip=True))
        #rec.append(elem.attrib["Body"])

        #elem.clear()
        #while elem.getprevious() is not None:
        #    del elem.getparent()[0]
        results.append(rec)

    return results

In [6]:
filename = "sofen_posts.xml"
chunksize = 1024

file_rdd = spark.read.text(filename, wholetext=True).rdd
dataset = file_rdd.flatMap(parse_post)

# Shingling

In [7]:
STOPWORDS: Final = stopwords.words('english')

def tokenize(text: str) -> List[str]:
    text = text.translate(str.maketrans('', '', string.punctuation))
    # text_nop = text.split()
    text_nop = word_tokenize(text)
    filtered_words = []

    for word in text_nop:
        if word not in STOPWORDS:
            filtered_words.append(word.lower())
        
    return filtered_words

    #def create_shingle(self, input_comment: comment_tuple, shingle_size: int) -> frozenset[tuple]:
    #    tokens = self.tokenize(input_comment.text)
    #    comment_length = len(tokens)
    #    shingles =  frozenset(tuple(tokens[i:(i + shingle_size)]) for i in range(comment_length - shingle_size + 1))
    
def create_shingle(post: str, shingle_size: int) -> list[list]:
    tokens = tokenize(post)
    comment_length = len(tokens)
    shingle_set = frozenset(hash(tuple(tokens[i:(i + shingle_size - 1)])) for i in range(comment_length - shingle_size))
    shingle_list = list(shingle_set)
    shingle_list.sort()
    return shingle_list
        

def shingle_map(row):
    return (row[0], create_shingle(row[1], SHINGLE_SIZE)
)

def set_shingle_schema():
    """
    Define the schema for the DataFrame
    """
    schema_list = []
    schema_list.append(StructField("Id", IntegerType(), True))
    #schema_list.append(StructField("PostTypeId", IntegerType(), True))
    #schema_list.append(StructField("Score", IntegerType(), True))
    schema_list.append(StructField("Shingles", ArrayType(ArrayType(StringType()), True)))
    return StructType(schema_list)

In [8]:
schema = set_shingle_schema()
shingle_rdd = dataset.map(shingle_map)

for elem in shingle_rdd.take(1):
    print(elem)

(1, [-7908626704179410037, -4684397193795377420, -3850838529138350569, -1419329986015534448, -473890435429154670, 1666811628384754881, 1736049749176541686, 2043274720988062366, 3773704842148420615, 6410835021690098780, 7162646048271668292, 7303886912402816729, 8156242454416184963])


# MinHashing

In [9]:
# Transform posts to characteristic matrix
# Make feature set matrix
# Minhash
# Make Minhash Matrix
# LSH
import random
SIGNATURE_SIZE: Final = 45
HASH_PRIME: Final = (1 << 31) - 1
MAX_HASH: Final = (1 << 32)
    #sys.maxsize
HASH_RANGE: Final = (1<< 32)
    #sys.maxsize
SEED: Final = 193120

generator = np.random.RandomState(SEED)
salts = [generator.randint(1, MAX_HASH) for _ in range(SIGNATURE_SIZE)]
#permutations = [generator.randint(1, HASH_PRIME) for _ in range(SIGNATURE_SIZE)]

permutations = [[generator.randint(1, MAX_HASH) for _ in range(SIGNATURE_SIZE)],
                [generator.randint(0, MAX_HASH) for _ in range(SIGNATURE_SIZE)]]

print(salts)
print(permutations)
    
def min_hasher(row):
    sig = np.full((SIGNATURE_SIZE), MAX_HASH)
    for shingle in row[1]:
        for i in range(SIGNATURE_SIZE):
            a = permutations[0][i]
            b = permutations[1][i]
            hash_val = (a * shingle + b) % HASH_PRIME
            sig[i] = min(hash_val, sig[i])
    return (row[0], sig)

[2098660572, 74013022, 2675907171, 2481944613, 2289028529, 846582864, 1024003631, 3457496236, 958218556, 2872479854, 1385329197, 2720560315, 2596604670, 4118717270, 3831528778, 4184700433, 130382780, 4221132295, 2492231677, 1349853675, 2674011478, 4230155413, 522369175, 1349933754, 2597169981, 1045439184, 3199517916, 3020468163, 618450142, 3282454786, 4061764145, 3477766529, 3055070885, 204747729, 834378094, 4046990333, 2141660100, 2118590568, 3537961224, 168082228, 652073352, 1284401985, 2826423363, 1325707607, 3731067674]
[[723716237, 3333080471, 2787841228, 2113795590, 3230829213, 1843969928, 746019385, 2309594694, 2229839123, 489336389, 779150137, 1446103079, 889348879, 3745936412, 4281307413, 2173665908, 1717438567, 23839969, 2145022294, 1052302893, 1118678081, 1295195691, 3851408124, 3928788718, 2370966349, 561561790, 1051546482, 4224882136, 4206633849, 1329396735, 1801554849, 1138195103, 3123692201, 1122989246, 1270364612, 1593839649, 1463959047, 4229718824, 1709163888, 33163415

In [10]:
a = permutations[0][1]
b = permutations[1][1]

hash_val = (a * hash(tuple(["text", "word", "shingle", "advanced"]))+ b) % HASH_PRIME
print(hash_val)

hash_val = (a * hash(tuple(["text", "word", "shingle", "advanced"]))+ b) % HASH_PRIME
print(hash_val)

1334347857
1334347857


In [11]:
hash_rdd = shingle_rdd.map(min_hasher)

for elem in hash_rdd.take(3):
    print(elem)

(1, array([845563453,  26442476, 133389465, 518734279,  23089274, 150791360,
         1629804,  12958004, 330303916, 162260333, 418509946, 167773993,
       123959452, 176671364,  35492587,  13045018,  42242984, 491448325,
       147130056, 422420168,  70311354, 480462515,  59325868, 207942073,
       495179390,  93101936,  51844004, 252573180,  18575239,   8717558,
        98433173, 168066857,  89221111,  45641238, 116367597,  11798369,
        79678701, 159035216,  74503017,  99454997,  45193347,  98964050,
        16470957, 279405817, 117088225]))
(3, array([ 53437723,   1807226,  46795477,  10543241,  73898747,    854791,
       144275303,  34492505,   3642539,   1115498,   6406941,  36984958,
        33538182, 105558336,  26441617,  24214184,  88258120,  56306856,
        11124257,  14818496,  21233719,  73332800,   2820214,  20051569,
        11686464,   4480021,   1489927,  98555934,  56450002,   7732484,
        61611242,  91514248,   2220041,  14732942,   7598738,   3431502,
 

# LSH

In [12]:
BANDS: Final = 15
ROWS: Final = 3
THRESHOLD: Final = (1/BANDS) ** (1/ROWS)
print(f"Bands: {BANDS}, rows {ROWS}, threshold {THRESHOLD}")

def hash_func(row):
    sum = 0
    for e in row[1][0]:
        sum += e
    return (row[0], (int.from_bytes(hashlib.md5(str(sum).encode()).digest()[:4], byteorder="big"), row[1][1]))

Bands: 15, rows 3, threshold 0.4054801330382267


In [13]:
# returns (doc, band, hash)
hash_band_rdd = hash_rdd.flatMap(lambda x: [[(x[0], i % BANDS), hash] for i, hash in enumerate(x[1])]).groupByKey()

for elem in hash_band_rdd.take(5):
    print(elem)

((39, 6), <pyspark.resultiterable.ResultIterable object at 0x7f711b688ca0>)
((54, 14), <pyspark.resultiterable.ResultIterable object at 0x7f711b688340>)
((68, 7), <pyspark.resultiterable.ResultIterable object at 0x7f711b688490>)
((81, 10), <pyspark.resultiterable.ResultIterable object at 0x7f711b6881c0>)
((110, 3), <pyspark.resultiterable.ResultIterable object at 0x7f711b688580>)


In [14]:
hash_bands_grouped_rdd = hash_band_rdd.map(lambda x: [x[0][1], (x[1], x[0][0])])

In [15]:
band_hashed = hash_bands_grouped_rdd.map(hash_func).map(lambda x: [(x[0], x[1][0]), x[1][1]]).groupByKey().filter(lambda x: (len(x[1]) > 1 and len(x[1]) < 50))

for elem in band_hashed.take(10):
    print(elem[0])
    for b in elem[1]:
        print(b)

(0, 1411846386)
167536
167535
(0, 4053313347)
350690
350691
(1, 1845215523)
41723
273035
(1, 3071030271)
344164
211443
(1, 3130031600)
162493
162492
(1, 3448823093)
232519
232518
(2, 238049773)
164686
324195
(3, 1014813404)
357131
153024
(3, 2489280772)
156840
355226
(3, 2665687479)
415563
148318


In [16]:
candidates = band_hashed.map(lambda x: (tuple(x[1]), 1)).reduceByKey(lambda a, b: ((float(a + b) / 10.0)))

for elem in candidates.take(100):
    print(elem)

((283945, 283955, 283943, 283947, 283950), 1)
((18158, 129614), 0.2)
((415033, 223891), 1)
((240253, 300699), 1)
((205545, 205544), 0.023111200000000002)
((130568, 214219), 0.031200000000000006)
((115363, 284707), 1)
((65456, 67115), 1)
((21857, 382291), 1)
((340982, 405789), 1)
((356045, 356043), 1)
((13642, 225220), 1)
((75867, 283663), 1)
((48542, 100102, 286200, 110928, 167600, 38460, 286198, 157129, 79090), 1)
((280280, 246106), 1)
((168072, 284774), 1)
((197065, 333939), 1)
((420957, 224129), 1)
((77048, 164490), 1)
((380008, 143259), 1)
((62890, 52315), 1)
((86553, 178286), 1)
((79090, 48542, 286200, 167600, 38460, 110928, 286198, 157129, 100102), 1)
((360086, 316921), 1)
((18079, 18074, 18080), 1)
((175742, 251288), 1)
((308443, 225917), 1)
((102201, 155247), 0.11120000000000001)
((379395, 305322), 1)
((237099, 252735), 1)
((396228, 423115), 1)
((91256, 412095), 1)
((179245, 179243), 0.11111111199999998)
((10738, 58433), 1)
((189796, 362316), 1)
((48542, 286200, 110928, 167600,

In [17]:
cand = candidates.collect()
shingle_dict = shingle_rdd.collectAsMap()

In [18]:
def calc_jaccard(list1, list2):
    return len(set(list1).intersection(list2)) / len(set(list1).union(list2))

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [19]:
import itertools

for elem in cand:
    #print(f"Candidates: {elem[0][0]}, {elem[0][1]}; similarity: ", end="")
    #print(f"{calc_jaccard(shingle_dict.get(elem[0][0]), shingle_dict.get(elem[0][1]))}")
    for comb in itertools.combinations(elem[0], 2):
        print(f"Candidates: {comb[0]}, {comb[1]}; similarity: ", end="")
        print(f"{calc_jaccard(shingle_dict.get(comb[0]), shingle_dict.get(comb[1]))}")

Candidates: 205545, 205544; similarity: 1.0
Candidates: 269513, 316271; similarity: 0.26666666666666666
Candidates: 197065, 333939; similarity: 0.0
Candidates: 420957, 224129; similarity: 0.0
Candidates: 77048, 164490; similarity: 0.0
Candidates: 18079, 18074; similarity: 0.09401709401709402
Candidates: 18079, 18080; similarity: 0.05806451612903226
Candidates: 18074, 18080; similarity: 0.12727272727272726
Candidates: 279342, 279302; similarity: 0.8095238095238095
Candidates: 234185, 234184; similarity: 0.6666666666666666
Candidates: 153241, 386812; similarity: 0.0
Candidates: 350691, 350690; similarity: 1.0
Candidates: 379395, 305322; similarity: 0.0
Candidates: 256121, 170983; similarity: 0.0
Candidates: 237099, 252735; similarity: 0.2903225806451613
Candidates: 91256, 412095; similarity: 0.0
Candidates: 316187, 15974; similarity: 0.0
Candidates: 232741, 232742; similarity: 0.3
Candidates: 31655, 82251; similarity: 0.0
Candidates: 236286, 210139; similarity: 0.0
Candidates: 10738, 584

# Exit Spark

In [20]:
spark.stop()